In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Custodian Accounts in LUSID

Attributes
----------
custodian accounts
transactions
transaction portfolios
"""

toggle_code("Toggle Docstring")

# Creating and using Custodian Accounts

This notebook demonstrates how to represent Custodian Accounts in LUSID and associate them with transactions. 

#### In this notebook: 
* We create property definitions in the Legal Entity and Custodian Account domains
* We model a Custodian as a Legal Entity
* We create a Custodian Account, linked to a Custodian
* We update the Custodian Account with additional properties
* We link the Custodian Account to transactions in a portfolio


## Setup

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as models

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken
from lusidtools.cocoon.utilities import generate_time_based_unique_id


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

In [3]:
# Define the APIs
transaction_portfolios_api = api_factory.build(lusid.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lusid.PortfoliosApi)
legal_entities_api = api_factory.build(lusid.LegalEntitiesApi)
property_definitions_api = api_factory.build(lusid.PropertyDefinitionsApi)

In [4]:
# Define scopes and Ids
scope = "custodian-account-testing"
portfolio_code = "test-portfolio"

custodian_id = "ID-123"
custodian_account_code_1 = "custodian-account-1"
custodian_account_code_2 = "custodian-account-2"

# Get sample transactions
file_name = "accounting-demo-sample-transactions.csv"
file_location = f"{os.getcwd()}/data/{file_name}"
sample_transactions = pd.read_csv(file_location, keep_default_na=False)

## 1. Create Custodian
A Custodian Account links directly to a Custodian, which is modelled as a Legal Entity.

### 1.1 Create Custodian Identifier Property
A Custodian is identified with an Identifier Property

In [5]:
try:
    property_definitions_api.create_property_definition(
        models.CreatePropertyDefinitionRequest(
            domain="LegalEntity",
            scope=scope,
            code="CustodianIdentifier",
            display_name="CustodianIdentifier",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            life_time="Perpetual",
            constraint_style="Identifier",
            property_description="Identifier for custodians",
        )
    )
    display(f"Created Property Definition'LegalEntity/{scope}/CustodianIdentifier' ")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

"Created Property Definition'LegalEntity/custodian-account-testing/CustodianIdentifier' "

### 1.2 Create a legal entity for the Custodian  

We create a single legal entity, using the Identifier property created earlier. 

In [6]:
try:
    response = legal_entities_api.upsert_legal_entity(
        models.UpsertLegalEntityRequest(
            identifiers={
                "LegalEntity/custodian-account-testing/CustodianIdentifier": lusid.ModelProperty(
                    key="LegalEntity/custodian-account-testing/CustodianIdentifier",
                    value=lusid.PropertyValue(label_value=custodian_id),
                )
            },
            display_name="GBPCustodian",
            description="Custodian for GBP Transactions",
        )
    )
    display(f"Created Custodian Legal Entity with ID {custodian_id}")
except lusid.ApiException as e:
    print(json.loads(e.body))

'Created Custodian Legal Entity with ID ID-123'

## 2. Create a Custodian Account

### 2.1 Create a Portfolio 
A Custodian Account is associated with a particular portfolio

In [7]:
try:
    response = transaction_portfolios_api.create_portfolio(
        scope,
        models.CreateTransactionPortfolioRequest(
            display_name="Portfolio Name",
            code=portfolio_code,
            base_currency="GBP",
            created="2018-01-01",
            sub_holding_keys=["Transaction/internal/ReferencedCustodianAccount"],
        ),
    )
    display(f"Created portfolio with scope {scope} and code {portfolio_code}")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

'Created portfolio with scope custodian-account-testing and code test-portfolio'

### 2.2 Upsert a Custodian Account
As we have created a Custodian, we can create a linked Custodian Account, with various fields to model information about the specific account. This Custodian Account is associated with a specific portfolio.

We use Upsert to create two Custodian Accounts, linked to the Custodian created earlier.

In [8]:
try:
    response = transaction_portfolios_api.upsert_custodian_accounts(
        scope,
        portfolio_code,
        [
            models.CustodianAccountRequest(
                code=custodian_account_code_1,
                account_number="1234",
                account_name="Default Custodian Account",
                accounting_method="Default",
                currency="GBP",
                custodian_identifier={
                    "idTypeScope": scope,
                    "idTypeCode": "CustodianIdentifier",
                    "code": custodian_id,
                },
            ),
            models.CustodianAccountRequest(
                code=custodian_account_code_2,
                account_number="5678",
                account_name="FIFO Custodian Account",
                accounting_method="FirstInFirstOut",
                currency="GBP",
                custodian_identifier={
                    "idTypeScope": scope,
                    "idTypeCode": "CustodianIdentifier",
                    "code": custodian_id,
                },
            ),
        ],
    )
    display(
        f"Created Custodian Accounts with codes {custodian_account_code_1} and {custodian_account_code_2}"
    )

except lusid.ApiException as e:
    print(json.loads(e.body))

'Created Custodian Accounts with codes custodian-account-1 and custodian-account-2'

### 2.3 Update properties on a Custodian Account
We can upsert individual properties onto an existing Custodian Account. The properties must be in the Custodian Account domain.

In [9]:
try:
    # Create a property definition in the Custodian Account domain
    property_definitions_api.create_property_definition(
        models.CreatePropertyDefinitionRequest(
            domain="CustodianAccount",
            scope=scope,
            code="Prop1",
            display_name="Prop1",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            life_time="TimeVariant",
            constraint_style="Property",
            property_description="Prop1",
        )
    )
    display(f"Created Property Definition 'CustodianAccount/{scope}/Prop1' ")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

try:
    response = transaction_portfolios_api.upsert_custodian_accounts_properties(
        scope,
        portfolio_code,
        scope,
        custodian_account_code_1,
        request_body={
            "CustodianAccount/custodian-account-testing/Prop1": lusid.ModelProperty(
                key="CustodianAccount/custodian-account-testing/Prop1",
                value=lusid.PropertyValue(label_value="Property1Valxyz"),
            )
        },
    )
    display(
        f"Upserted Property 'CustodianAccount/{scope}/Prop1' onto Custodian Account with code {custodian_account_code_1}"
    )
except lusid.ApiException as e:
    print(json.loads(e.body))

"Created Property Definition 'CustodianAccount/custodian-account-testing/Prop1' "

"Upserted Property 'CustodianAccount/custodian-account-testing/Prop1' onto Custodian Account with code custodian-account-1"

## 3. View Custodian Accounts

### 3.1 List the custodian accounts for a given portfolio
We can retrieve all Custodian Accounts that are associated with a portfolio

In [10]:
list_custodian_accounts_response = lusid_response_to_data_frame(
    transaction_portfolios_api.list_custodian_accounts(scope, portfolio_code),
    rename_properties=True,
)

# Rename columns for clarity
columnsToRename = {
    "custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.key": "Custodian Identifier Property Key",
    "custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.value.label_value": "Custodian Identifier Property Value",
}
list_custodian_accounts_response.rename(
    columns=columnsToRename,
    inplace=True,
)


columnsToDrop = [
    "custodian.href",
    "custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.effective_from",
    "custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.effective_until",
    "custodian.relationships",
    "custodian.version.effective_from",
    "custodian.version.as_at_date",
    "account_type",
    "properties",
]
# Display response as a dataframe, with some columns removed for brevity
display(list_custodian_accounts_response.drop(columns=columnsToDrop))

,custodian_account_id.scope,custodian_account_id.code,status,account_number,account_name,accounting_method,currency,Prop1(custodian-account-testing-Properties),custodian.display_name,custodian.description,custodian.lusid_legal_entity_id,Custodian Identifier Property Key,Custodian Identifier Property Value,custodian.properties
0,custodian-account-testing,custodian-account-1,Active,1234,Default Custodian Account,Default,GBP,Property1Valxyz,GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,{}
1,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,NaN,GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,{}


### 3.2 Get a specific Custodian Account
We can retrieve a certain Custodian Account in a portfolio, using the Custodian Account scope and code.

In [11]:
get_custodian_account_response = lusid_response_to_data_frame(
    transaction_portfolios_api.get_custodian_account(
        scope, portfolio_code, scope, custodian_account_code_1
    ),
    rename_properties=True,
).T

# Rename columns for clarity
columnsToRename = {
    "custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.key": "Custodian Identifier Property Key",
    "custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.value.label_value": "Custodian Identifier Property Value",
    "properties.CustodianAccount/custodian-account-testing/Prop1.key": "Property Key",
    "properties.CustodianAccount/custodian-account-testing/Prop1.value.label_value": "Property Value",
}
get_custodian_account_response.rename(columns=columnsToRename, inplace=True)

columnsToDrop = [
    "properties.CustodianAccount/custodian-account-testing/Prop1.value.metric_value",
    "properties.CustodianAccount/custodian-account-testing/Prop1.value.label_value_set",
    "properties.CustodianAccount/custodian-account-testing/Prop1.effective_from",
    "properties.CustodianAccount/custodian-account-testing/Prop1.effective_until",
    "custodian.href",
]
# Display response, with some columns removed for brevity
display(get_custodian_account_response.iloc[:, :19].drop(columns=columnsToDrop))

,custodian_account_id.scope,custodian_account_id.code,status,account_number,account_name,accounting_method,currency,Property Key,Property Value,custodian.display_name,custodian.description,custodian.lusid_legal_entity_id,Custodian Identifier Property Key,Custodian Identifier Property Value
response_values,custodian-account-testing,custodian-account-1,Active,1234,Default Custodian Account,Default,GBP,CustodianAccount/custodian-account-testing/Prop1,Property1Valxyz,GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123


## 4. Using Custodian Accounts with transactions

### 4.1 Upsert transactions with associated custodian accounts 
We can add transactions to our portfolio linked to an existing Custodian Account, using the __custodian_account_id__ field on the transaction request.

Custodian Accounts can also be added to existing transactions.

In [12]:
# Read transactions from a CSV and transform into LUSID transactions
transactions_to_upsert = []
for index, txn in sample_transactions.iterrows():
    if txn["desc"] != "FundsIn":
        transactions_to_upsert.append(
            models.TransactionRequest(
                transaction_id=txn["t_id"],
                type="Simple&Sell",
                instrument_identifiers={
                    "Instrument/default/Figi": txn["figi_identifier"]
                },
                transaction_date=txn["trade_datetime"],
                settlement_date=txn["settlement_datetime"],
                units=txn["qty"],
                transaction_price=models.TransactionPrice(
                    price=txn["unit_price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    # Calculate cost on-the-fly
                    amount=txn["qty"] * txn["unit_price"],
                    currency=txn["trade_currency"],
                ),
                # Specify custodian account depending on the transaction unit price
                custodian_account_id={
                    "scope": scope,
                    "code": (
                        custodian_account_code_1
                        if txn["unit_price"] < 5
                        else custodian_account_code_2
                    ),
                },
                source="mysource",
            )
        )

try:
    response = transaction_portfolios_api.upsert_transactions(
        scope, portfolio_code, transactions_to_upsert
    )
    display("Transactions upserted")

except lusid.ApiException as e:
    print(json.loads(e.body))

'Transactions upserted'

### 4.2 Retrieve transactions with Custodian Account information
When we retrieve the transaction, the __custodian_account__ field will be populated with Custodian Account information from the associated ID. We can request specific Custodian Account properties to be returned on the transactions, else all Custodian Account property keys will be returned.

In [13]:
# Get the transactions and display the custodian related fields
get_transactions_response = lusid_response_to_data_frame(
    transaction_portfolios_api.get_transactions(scope, portfolio_code),
    rename_properties=True,
)
custodian_columns = ["transaction_id"] + [
    col for col in get_transactions_response if col.startswith("custodian")
]
display(get_transactions_response[custodian_columns])

,transaction_id,custodian_account.custodian_account_id.scope,custodian_account.custodian_account_id.code,custodian_account.status,custodian_account.account_number,custodian_account.account_name,custodian_account.accounting_method,custodian_account.currency,custodian_account.properties,custodian_account.custodian.display_name,custodian_account.custodian.description,custodian_account.custodian.lusid_legal_entity_id,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.key,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.value.label_value,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.effective_from,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.effective_until,custodian_account.custodian.properties,custodian_account.custodian.relationships,custodian_account.custodian.version.effective_from,custodian_account.custodian.version.as_at_date,custodian_account.account_type,custodian_account.properties.CustodianAccount/custodian-account-testing/Prop1.key,custodian_account.properties.CustodianAccount/custodian-account-testing/Prop1.value.label_value,custodian_account.properties.CustodianAccount/custodian-account-testing/Prop1.effective_from
0,435opkplk03sfsf33,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin,NaN,NaN,NaN
1,dsfsdlkjs933234211,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin,NaN,NaN,NaN
2,dfklsjd90232350239,custodian-account-testing,custodian-account-1,Active,1234,Default Custodian Account,Default,GBP,NaN,GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin,CustodianAccount/custodian-account-testing/Prop1,Property1Valxyz,0001-01-01 00:00:00+00:00
3,kj0llk90f0909201141,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin,NaN,NaN,NaN
4,sadjkas934234sadfs,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin,NaN,NaN,NaN
5,fas2340090sd9f0s23,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin,NaN,NaN,NaN
6,dflkjsdflk932432rsd,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+

### 4.3 Filter transactions by Custodian Account
We can filter the transactions based on an attribute of the associated Custodian Account, using the __filter__ field in the __get_transactions__ API call.

In [14]:
# Filter the transactions where Custodian Account Account Name is "FIFO Custodian Account"
get_transactions_response = lusid_response_to_data_frame(
    transaction_portfolios_api.get_transactions(
        scope=scope,
        code=portfolio_code,
        filter="CustodianAccount.AccountName eq 'FIFO custodian account'",
    ),
    rename_properties=True,
)
custodian_columns = ["transaction_id"] + [
    col for col in get_transactions_response if col.startswith("custodian")
]
display(get_transactions_response[custodian_columns])

,transaction_id,custodian_account.custodian_account_id.scope,custodian_account.custodian_account_id.code,custodian_account.status,custodian_account.account_number,custodian_account.account_name,custodian_account.accounting_method,custodian_account.currency,custodian_account.properties,custodian_account.custodian.display_name,custodian_account.custodian.description,custodian_account.custodian.lusid_legal_entity_id,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.key,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.value.label_value,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.effective_from,custodian_account.custodian.identifiers.LegalEntity/custodian-account-testing/CustodianIdentifier.effective_until,custodian_account.custodian.properties,custodian_account.custodian.relationships,custodian_account.custodian.version.effective_from,custodian_account.custodian.version.as_at_date,custodian_account.account_type
0,435opkplk03sfsf33,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin
1,dsfsdlkjs933234211,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin
2,kj0llk90f0909201141,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin
3,sadjkas934234sadfs,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin
4,fas2340090sd9f0s23,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin
5,dflkjsdflk932432rsd,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin
6,2354lafjaso3432502,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin
7,sdlfksdlfk23409235,custodian-account-testing,custodian-account-2,Active,5678,FIFO Custodian Account,FirstInFirstOut,GBP,{},GBPCustodian,Custodian for GBP Transactions,LUID_00003D6A,LegalEntity/custodian-account-testing/Custodia...,ID-123,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,{},[],0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,Margin


## Clean up 

In [15]:
try:
    transaction_portfolios_api.delete_custodian_accounts(
        scope,
        portfolio_code,
        [{"scope": scope, "code": custodian_account_code_1}],
        delete_mode="Hard",
    )
    transaction_portfolios_api.delete_custodian_accounts(
        scope,
        portfolio_code,
        [{"scope": scope, "code": custodian_account_code_2}],
        delete_mode="Hard",
    )
    legal_entities_api.delete_legal_entity(scope, "CustodianIdentifier", custodian_id)
    property_definitions_api.delete_property_definition(
        "LegalEntity", scope, "CustodianIdentifier"
    )
    property_definitions_api.delete_property_definition(
        "CustodianAccount", scope, "Prop1"
    )
    portfolios_api.delete_portfolio(scope, portfolio_code)

except lusid.ApiException as e:
    print(json.loads(e.body))